# Batch Process Source HEC-RAS Models
Generate FIM Libraries for multiple HEC-RAS Models

In [1]:
%load_ext autoreload
%autoreload 2

from src import *
from src.setup import *
from src.process import *

### `Parameters`

In [ ]:
collection_name = "" # Define Collection name - example: "ble_12100302_Medina"

### Instantiate CollectionData, Database, and JobClient objects

In [2]:
collection = CollectionData(collection_name)
database = Database(collection)
jobclient = JobClient(collection)

### Get Working Models

In [3]:
# Obtain models data from what is currently on disk. 
model_ids = collection.get_models()

### Run `conflate_model`

In [4]:
conflate_model_batch = ConflateModelBatchProcessor(collection, model_ids) 
conflate_model_batch.conflate_model_batch_process(jobclient, database)

### Load Conflation Information to Database

In [5]:
succeeded_and_unknown_status_models = [
    model_id[0]
    for model_id in conflate_model_batch.succeeded + conflate_model_batch.unknown
]
load_conflation(succeeded_and_unknown_status_models, database)

### Update `network_to_id` Table in Database
Build a modified network by skipping over reaches eclipsed by adjacent reaches

In [6]:
update_network(database)

### Get Working Reaches

In [7]:
reaches = database.get_reaches_by_models(model_ids)
reach_data = [(data[0], data[2]) for data in reaches]

### Instantiate reach_processor 

In [8]:
reach_step_processor = ReachStepProcessor(collection, reach_data)

### Run `extract_submodel` step

In [9]:
reach_step_processor.execute_extract_submodel_process(jobclient, database, timeout=5)

### Run `create_ras_terrain` step

In [10]:
reach_step_processor.execute_process(jobclient, database, "create_ras_terrain", timeout=3)

### Run `create_model_run_normal_depth` step

In [11]:
reach_step_processor.execute_process(jobclient, database, "create_model_run_normal_depth", timeout=10)

### Run `run_incremental_normal_depth` step

In [12]:
reach_step_processor.execute_process(jobclient, database, "run_incremental_normal_depth", timeout=25)

### Initial `run_known_wse` and Initial `create_rating_curves_db`

In [13]:
outlet_reaches = [data[0] for data in reaches if data[1] is None]
execute_ikwse_for_network(
    [(reach, None) for reach in outlet_reaches],
    collection,
    database,
    jobclient,
    False,
    timeout=20,
)

### Final `run_known_wse`

In [14]:
kwse_reach_data = [
    (data[0], data[1])
    for data in reaches
    if data[1] is not None
    and data[0]
    in reach_step_processor.succesful_and_unknown_reaches
]

kwse_step_processor = KWSEStepProcessor(collection, kwse_reach_data)

kwse_step_processor.execute_kwse_step(jobclient, database, "run_known_wse", timeout=180)

In [ ]:
# TODO Refactor below:
#  
# # Optional if KWSE is not performed
# succeded_reaches, failed_reaches, not_accepted_reaches, unknown_status_reaches = execute_step([(reach[0], "") for reach in succeded_reaches + unknown_status_reaches], "create_fim_lib", db_path, source_models_dir, submodels_dir, library_dir, timeout_minutes=20)
# if stop_on_error and (len(failed_reaches) + len(not_accepted_reaches) + len(unknown_status_reaches)) > 0:
#     raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")
# outlet_reaches = [data[0] for data in reach_data if data[1] is None]

### Final `create_rating_curves_db` step

In [15]:
reach_step_processor.execute_process(jobclient, database, "create_rating_curves_db", timeout=15)

### Merge Rating Curves

In [16]:
load_all_rating_curves(database)

### Run `create_fim_lib` Step

In [17]:
reach_step_processor.execute_process(jobclient, database, "create_fim_lib", timeout=30)

### Create extent library step

In [18]:
create_extent_lib(collection)

### Create Flows2FIM Start Reaches File

In [19]:
outlet_reaches = [data[0] for data in reaches if data[1] is None]
create_f2f_start_file(outlet_reaches, collection.f2f_start_file)

In [16]:
# !jupyter nbconvert --to html "process copy 2.ipynb"